# FFPP Deepfake Detection Pipeline (80:20 → 7:1 → 5-Fold → Test)

이 노트북은 “alltype” 폴더 구조를 사용하는 FaceForensics++ 데이터셋을 기반으로,  
아래 절차를 순차적으로 수행합니다:

1. **데이터 로드 및 전처리 확인**  
   - `configs/ffpp_c23.yaml`을 기준으로 데이터 경로(`dataset_root/version/alltype`) 설정  
   - `FFPPFrameDataset`을 사용해 모든 프레임 이미지와 레이블(Real/Fake) 로드  
   - 전처리 파이프라인(`get_ffpp_transforms`)이 정상 작동하는지 샘플 이미지 시각화

2. **Train/Val/Test 분할 및 5-Fold Cross-Validation 학습**  
   - 전체(alltype) 데이터 → 80% Train+Val / 20% Test 분할 (stratify)  
   - Train+Val(80%) → Train(70%) / Val(10%) 분할  
   - Train(70%) → 5-Fold Cross-Validation (Train:Val = 4:1)  
   - 각 Fold마다 학습 및 검증 수행 → Fold별 “best.pth” 저장

3. **Test(20%) 성능 평가**  
   - 마지막(5번째) Fold의 “best.pth” 모델을 불러와 Test 데이터셋에 대해 AUC/EER 계산

4. **단일 비디오 추론 (Inference)**  
   - Test 평가 이후, 새로운 비디오 파일을 “영상 → 프레임”으로 분할  
   - 각 프레임별 Deepfake 확률 예측 → JSON으로 저장  
   - (선택) 상위 확률 프레임 5개 시각화

---

**사전 준비**  
1. `deepfake_adapter_ffpp/` 디렉터리 구조가 다음과 같아야 합니다:
    ```
    deepfake_adapter_ffpp/
    ├── configs/
    │   ├── ffpp_c23.yaml
    │   └── ffpp_c40.yaml
    ├── data/
    │   └── README.md
    ├── notebooks/
    │   └── 01_pipeline.ipynb
    ├── src/
    │   ├── datasets/ffpp_dataset.py
    │   ├── models/
    │   │   ├── backbone.py
    │   │   ├── adapters.py
    │   │   └── deepfake_adapter.py
    │   ├── utils/
    │   │   ├── transforms.py
    │   │   └── metrics.py
    │   └── scripts/
    │       ├── train.py
    │       └── predict.py
    ├── outputs/
    │   ├── checkpoints/
    │   └── logs/
    ├── requirements.txt
    └── README.md
    ```
2. `configs/ffpp_c23.yaml`의 `dataset_root` 경로를 본인 환경에 맞게 수정하세요.  
   예: `"dataset_root": "D:/Files/data/FaceForensicspp_RECCE"`  
3. 필요한 라이브러리를 설치합니다:  
   ```bash
   pip install -r requirements.txt


- `configs/ffpp_c23.yaml`의 `dataset_root` 경로를 본인 환경에 맞게 수정하세요.  
- 필요한 라이브러리(`torch`, `timm`, `yaml`, `opencv-python`, `torchvision`, `sklearn` 등)를 설치합니다:  
```bash
pip install -r requirements.txt


In [1]:
# 셀 2: 라이브러리 임포트 및 설정 불러오기

import os
import yaml
import random
import json
import numpy as np
import torch
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
from PIL import Image
import cv2
import matplotlib.pyplot as plt

# src/ 경로를 import 경로에 추가
import sys
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))

# 사용자 정의 모듈
from src.datasets.ffpp_dataset import FFPPFrameDataset
from src.models.deepfake_adapter import DeepfakeAdapter
from src.utils.transforms import get_ffpp_transforms
from src.utils.metrics import compute_auc, compute_eer

# --- 설정(Config) 로드 ---
config_path = "../configs/ffpp_c23.yaml"  # 노트북이 notebooks/ 폴더 안에 있으므로 상위 폴더 참조
with open(config_path, "r") as f:
    cfg = yaml.safe_load(f)

# 재현성(Seed) 고정 함수
def set_seed(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

# 디바이스 설정 (GPU 사용 우선)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Device: {device}")
print("Config:", cfg)


Device: cuda
Config: {'dataset_root': 'D:/My Project/Deepfake/df_train/data/FaceForensicspp_RECCE', 'version': 'c23_frames', 'input_size': 224, 'batch_size': 32, 'num_workers': 4, 'model': {'backbone': 'vit_base_patch16_224', 'adapter_dim': 128, 'num_classes': 2}, 'optimizer': {'type': 'adamw', 'lr': '1e-4', 'weight_decay': '1e-5'}, 'scheduler': {'type': 'cosine', 'T_max': 50}, 'training': {'epochs': 15, 'k_folds': 5}, 'output_dir': 'outputs/checkpoints/ffpp_c23', 'log_dir': 'outputs/logs/ffpp_c23'}


## 데이터셋 로드 및 전처리 확인

- `FFPPFrameDataset`을 이용해 **alltype** 데이터 전체를 로드합니다.
- `get_ffpp_transforms(cfg["input_size"])`으로 이미지가 Resize → ToTensor → Normalize 되는지 확인합니다.
- 샘플 이미지를 시각화하여 전처리 결과를 눈으로 확인해 봅니다.


In [2]:
# 셀 3: 데이터셋 로드 및 전처리 확인 (샘플링, 증강 포함)

set_seed(42)

# (A) 학습/검증용 transform 불러오기
train_transform, val_transform = get_ffpp_transforms(cfg["input_size"])

# (B) 전체 데이터셋 로드 (샘플링 interval=5, transform=None → 단순 count용)
full_dataset = FFPPFrameDataset(
    config_path,
    frame_interval=5,   # 5프레임당 1샘플 사용
    transform=None      # 단순 통계용이므로 transform은 생략
)
print(f"Total sampled samples (alltype): {len(full_dataset)}")

# 레이블별 개수 세기
labels_array = np.array(full_dataset.labels)
unique, counts = np.unique(labels_array, return_counts=True)
label_counts = dict(zip(unique, counts))
print(f"Label counts (샘플링 후): {label_counts}  (0=real, 1=fake)")

# (C) 임시 DataLoader 구성 — 배치 형태 확인용
# 여기서는 학습용 transform만 적용해서 한번 이미지가 어떻게 변하는지 시각화해보고 싶다면:
vis_dataset = FFPPFrameDataset(
    config_path,
    frame_interval=5,
    transform=train_transform
)
temp_loader = DataLoader(
    vis_dataset,
    batch_size=cfg["batch_size"],
    shuffle=True,
    num_workers=cfg["num_workers"],
    pin_memory=True
)

# (D) 첫 번째 배치 가져와서 이미지 텐서 모양 확인
images, labels = next(iter(temp_loader))
print("Batch images shape:", images.shape)  # [batch_size, 3, input_size, input_size]
print("Batch labels shape:", labels.shape)  # [batch_size]

# (E) 샘플 이미지 시각화 (정규화 해제 후 표시)
def imshow_tensor(img_tensor, mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]):
    """정규화된 텐서를 이미지로 되돌려 Matplotlib으로 표시"""
    img = img_tensor.clone().cpu().numpy()
    for c in range(3):
        img[c] = img[c] * std[c] + mean[c]
    img = np.clip(img, 0, 1)
    img = np.transpose(img, (1, 2, 0))  # (H, W, C)
    plt.imshow(img)
    plt.axis("off")

fig, axes = plt.subplots(1, 4, figsize=(12, 4))
for i in range(4):
    imshow_tensor(images[i])
    axes[i].set_title(f"Label: {labels[i].item()}")
plt.show()


Total sampled samples (alltype): 430850
Label counts (샘플링 후): {np.int64(0): np.int64(247008), np.int64(1): np.int64(183842)}  (0=real, 1=fake)


KeyboardInterrupt: 

## Train/Val/Test 분할 (80:20 → 7:1) + 5-Fold 학습/검증 + Test 평가

1. 전체(alltype) 데이터 → Train+Val(80%) / Test(20%) 분할 (stratify)
2. Train+Val(80%) → Train(70%) / Val(10%) 분할 (stratify)
3. Train(70%)에 대해 **5-Fold Cross-Validation** 수행
   - 각 Fold마다 Train/Val 로더 구성 → 학습 및 검증 → Fold별 최적 모델 저장
4. 마지막(5번째) Fold의 모델을 불러와 Test(20%) 평가 (AUC/EER)


In [ ]:
# 셀 4: Train/Val/Test 분할 + 5-Fold 학습/검증 + Test 평가 (샘플링·증강·불균형 보정 + Early Stopping)

from sklearn.model_selection import train_test_split, KFold
from torch.utils.data import Subset, WeightedRandomSampler
from tqdm import tqdm

set_seed(42)

# (1) 전체 데이터셋 로드 (샘플링 interval=5, transform=None → 분할용)
base_dataset = FFPPFrameDataset(
    config_path,
    frame_interval=5,   # 프레임 샘플링 비율
    transform=None      # 분할 통계만 필요하므로 transform 생략
)
N = len(base_dataset)
print(f"Total sampled samples (alltype): {N}")

# (2) 레이블 배열/분포 확인 (이미 셀 3에서 했지만, 다시 한 번)
all_labels = list(base_dataset.labels)
unique, counts = np.unique(np.array(all_labels), return_counts=True)
print(f"전체 레이블 분포: {dict(zip(unique, counts))} (0=real,1=fake)")

# (3) 80:20 Train+Val / Test 분할 (stratify)
all_indices = list(range(N))
trainval_idx, test_idx = train_test_split(
    all_indices,
    test_size=0.2,
    random_state=42,
    stratify=all_labels
)
print(f"Train+Val count: {len(trainval_idx)}, Test count: {len(test_idx)}")

# (4) Train+Val(80%) → Train(70%) / Val(10%) 분할 (stratify)
train_idx, val_idx = train_test_split(
    trainval_idx,
    test_size=0.125,  # 0.125 * 0.8 = 0.10 전체
    random_state=42,
    stratify=[all_labels[i] for i in trainval_idx]
)
print(f"Train count: {len(train_idx)}, Val count: {len(val_idx)}, Test count: {len(test_idx)}")

# (5) 클래스 불균형 보정을 위한 가중치 계산
counts_train_labels = [all_labels[i] for i in train_idx]
unique_tr, counts_tr = np.unique(np.array(counts_train_labels), return_counts=True)
class_counts = dict(zip(unique_tr, counts_tr))  # {0: real_count, 1: fake_count}
total_train = sum(class_counts.values())
# 가중치 = 전체 샘플 수 / (클래스별 샘플 수)
weight_per_label = {
    0: total_train / class_counts[0],
    1: total_train / class_counts[1]
}
print(f"Train set 클래스별 카운트: {class_counts}, 가중치: {weight_per_label}")

# (6) 5-Fold 학습/검증
k_folds = cfg["training"].get("k_folds", 5)
kfold = KFold(n_splits=k_folds, shuffle=True, random_state=42)

# Early Stopping 기준
patience = 3  # Validation AUC가 3회 연속 개선되지 않으면 멈춤

# 학습/검증 loop
for fold, (tr_sub, va_sub) in enumerate(kfold.split(train_idx)):
    print(f"\n--- Fold {fold+1}/{k_folds} ---")
    # (A) 실제 Train/Val 인덱스 매핑
    true_train_idx = [train_idx[i] for i in tr_sub]
    true_val_idx   = [train_idx[i] for i in va_sub]

    # (B) Dataset 객체 생성 (샘플링 interval=5 + transform 적용 + 부분 인덱스)
    train_dataset = FFPPFrameDataset(
        config_path,
        frame_interval=5,
        transform=train_transform,    # 학습용 증강
        indices=true_train_idx
    )
    val_dataset   = FFPPFrameDataset(
        config_path,
        frame_interval=5,
        transform=val_transform,      # 검증용 최소 전처리
        indices=true_val_idx
    )

    # (C) WeightedRandomSampler 구성 (클래스 불균형 보정)
    train_labels_for_sampler = [train_dataset.labels[i] for i in range(len(train_dataset))]
    sample_weights = [weight_per_label[label] for label in train_labels_for_sampler]
    sampler = WeightedRandomSampler(
        sample_weights,
        num_samples=len(sample_weights),
        replacement=True
    )

    # (D) DataLoader 구성
    train_loader = DataLoader(
        train_dataset,
        batch_size=cfg["batch_size"],
        sampler=sampler,            # shuffle=False 대신 sampler 사용
        num_workers=cfg["num_workers"],
        pin_memory=True
    )
    val_loader = DataLoader(
        val_dataset,
        batch_size=cfg["batch_size"],
        shuffle=False,
        num_workers=cfg["num_workers"],
        pin_memory=True
    )

    # (E) 모델/손실함수/옵티/스케줄러 설정
    model = DeepfakeAdapter(cfg).to(device)
    # CrossEntropyLoss 대신 class_weights 적용
    class_weights_tensor = torch.tensor([weight_per_label[0], weight_per_label[1]],
                                        dtype=torch.float).to(device)
    criterion = torch.nn.CrossEntropyLoss(weight=class_weights_tensor)
    optimizer = torch.optim.AdamW(
        model.parameters(),
        lr=float(cfg["optimizer"]["lr"]),
        weight_decay=float(cfg["optimizer"]["weight_decay"])
    )
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
        optimizer, T_max=int(cfg["scheduler"]["T_max"])
    )

    # (F) Fold별 체크포인트/로그 디렉터리 생성
    fold_ckpt_dir = os.path.join(cfg["output_dir"], f"fold_{fold+1}")
    fold_log_dir  = os.path.join(cfg["log_dir"],  f"fold_{fold+1}")
    os.makedirs(fold_ckpt_dir, exist_ok=True)
    os.makedirs(fold_log_dir, exist_ok=True)

    writer = SummaryWriter(log_dir=fold_log_dir)
    best_auc = 0.0
    epochs_since_improve = 0

    max_epochs = int(cfg["training"]["epochs"])
    for epoch in range(max_epochs):
        # (G-1) Training 단계
        model.train()
        running_loss = 0.0
        for images, labels in tqdm(train_loader,
                                   desc=f"Fold {fold+1} Train Epoch {epoch+1}",
                                   leave=False):
            images = images.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()
            logits = model(images)
            loss = criterion(logits, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * images.size(0)

        scheduler.step()
        avg_train_loss = running_loss / len(train_loader.dataset)
        writer.add_scalar("Loss/train", avg_train_loss, epoch)

        # (G-2) Validation 단계
        model.eval()
        all_labels_fold = []
        all_scores_fold = []
        with torch.no_grad():
            for images, labels in tqdm(val_loader,
                                       desc=f"Fold {fold+1} Val Epoch {epoch+1}",
                                       leave=False):
                images = images.to(device)
                labels = labels.to(device)
                logits = model(images)
                probs = torch.softmax(logits, dim=1)[:, 1]  # fake 확률
                all_labels_fold.extend(labels.cpu().numpy())
                all_scores_fold.extend(probs.cpu().numpy())

        val_auc = compute_auc(np.array(all_labels_fold), np.array(all_scores_fold))
        val_eer = compute_eer(np.array(all_labels_fold), np.array(all_scores_fold))
        writer.add_scalar("AUC/val", val_auc, epoch)
        writer.add_scalar("EER/val", val_eer, epoch)

        print(
            f"[Fold {fold+1}/{k_folds} | Epoch {epoch+1}/{max_epochs}] "
            f"Train Loss: {avg_train_loss:.4f}  Val AUC: {val_auc:.4f}  Val EER: {val_eer:.4f}"
        )

        # (G-3) 최적 모델 저장 및 Early Stopping 체크
        if val_auc > best_auc:
            best_auc = val_auc
            epochs_since_improve = 0
            ckpt = {
                "epoch": epoch,
                "model_state_dict": model.state_dict(),
                "optimizer_state_dict": optimizer.state_dict(),
                "auc": val_auc,
            }
            save_path = os.path.join(fold_ckpt_dir, "best.pth")
            torch.save(ckpt, save_path)
            print(f"→ New best model for Fold {fold+1} saved at {save_path}")
        else:
            epochs_since_improve += 1

        # patience Epoch 연속 개선 없으면 조기 종료
        if epochs_since_improve >= patience:
            print(f"→ Fold {fold+1}: {patience} Epoch 연속 Val AUC 개선 없음 → 조기 종료 (Epoch {epoch+1})")
            break

    writer.close()

# (H) 5-Fold 완료 후 Test 셋 평가 (transform=val_transform)
print("\n=== K-Fold Training Complete ===")
print("Evaluating on the Test set using Fold 5's best model...")

test_dataset = FFPPFrameDataset(
    config_path,
    frame_interval=5,
    transform=val_transform,
    indices=test_idx
)
test_loader  = DataLoader(
    test_dataset,
    batch_size=cfg["batch_size"],
    shuffle=False,
    num_workers=cfg["num_workers"],
    pin_memory=True
)
print(f"Test samples: {len(test_dataset)}")

last_ckpt_path = os.path.join(cfg["output_dir"], f"fold_{k_folds}", "best.pth")
assert os.path.exists(last_ckpt_path), f"Checkpoint not found: {last_ckpt_path}"

model = DeepfakeAdapter(cfg).to(device)
ckpt = torch.load(last_ckpt_path, map_location=device)
model.load_state_dict(ckpt["model_state_dict"])
# (H) Test 평가
model.eval()
all_test_labels = []
all_test_scores = []
with torch.no_grad():  # Test 루프도 no_grad로 묶기
    for images, labels in tqdm(test_loader, desc="Test Evaluation", leave=False):
        images = images.to(device)
        labels = labels.to(device)
        logits = model(images)
        probs = torch.softmax(logits, dim=1)[:, 1]
        all_test_labels.extend(labels.cpu().numpy())
        all_test_scores.extend(probs.cpu().numpy())

test_auc = compute_auc(np.array(all_test_labels), np.array(all_test_scores))
test_eer = compute_eer(np.array(all_test_labels), np.array(all_test_scores))
print(f"Final Test AUC: {test_auc:.4f}, Test EER: {test_eer:.4f}")


Total sampled samples (alltype): 430850
전체 레이블 분포: {np.int64(0): np.int64(247008), np.int64(1): np.int64(183842)} (0=real,1=fake)
Train+Val count: 344680, Test count: 86170
Train count: 301595, Val count: 43085, Test count: 86170
Train set 클래스별 카운트: {np.int64(0): np.int64(172905), np.int64(1): np.int64(128690)}, 가중치: {0: np.float64(1.7442815418871636), 1: np.float64(2.3435775895562982)}

--- Fold 1/5 ---


[Fold 1/5 | Epoch 1/15] Train Loss: 0.3554  Val AUC: 0.9815  Val EER: 0.0767
→ New best model for Fold 1 saved at outputs/checkpoints/ffpp_c23\fold_1\best.pth


[Fold 1/5 | Epoch 2/15] Train Loss: 0.1869  Val AUC: 0.9932  Val EER: 0.0454
→ New best model for Fold 1 saved at outputs/checkpoints/ffpp_c23\fold_1\best.pth


[Fold 1/5 | Epoch 3/15] Train Loss: 0.1368  Val AUC: 0.9964  Val EER: 0.0332
→ New best model for Fold 1 saved at outputs/checkpoints/ffpp_c23\fold_1\best.pth


[Fold 1/5 | Epoch 4/15] Train Loss: 0.1110  Val AUC: 0.9979  Val EER: 0.0260
→ New best model for Fold 1 saved at outputs/checkpoints/ffpp_c23\fold_1\best.pth


...

[Fold 1/5 | Epoch 6/15] Train Loss: 0.0812  Val AUC: 0.9986  Val EER: 0.0218
→ New best model for Fold 1 saved at outputs/checkpoints/ffpp_c23\fold_1\best.pth


[Fold 1/5 | Epoch 7/15] Train Loss: 0.0724  Val AUC: 0.9988  Val EER: 0.0202
→ New best model for Fold 1 saved at outputs/checkpoints/ffpp_c23\fold_1\best.pth


[Fold 1/5 | Epoch 8/15] Train Loss: 0.0673  Val AUC: 0.9992  Val EER: 0.0175
→ New best model for Fold 1 saved at outputs/checkpoints/ffpp_c23\fold_1\best.pth


[Fold 1/5 | Epoch 9/15] Train Loss: 0.0617  Val AUC: 0.9991  Val EER: 0.0186


KeyboardInterrupt: 

## 추론(Inference)

- 학습된 Fold 5 모델(`fold_5/best.pth`)을 사용하여 **단일 비디오**에 대해 Deepfake 확률을 예측하고,  
  결과를 프레임 단위 JSON 파일(`outputs/predictions/[video_name].json`)로 저장합니다.
- 과정:
  1. `cv2.VideoCapture`로 비디오를 열어 `temp_frames_infer/` 폴더에 프레임 저장  
  2. 저장된 프레임에 대해 `get_ffpp_transforms(cfg["input_size"])` 적용 후 모델 예측  
  3. 각 프레임별 fake 확률을 JSON으로 저장


In [ ]:
# 셀 5: 추론(Inference)

# (A) 추론할 비디오 파일 경로 지정 (본인이 테스트할 비디오 경로로 수정)
test_video_path = "../data/raw_videos/sample_video.mp4"  # 예시
assert os.path.exists(test_video_path), f"Video not found: {test_video_path}"

# (B) 임시 폴더에 프레임 추출
temp_frames_dir = "temp_frames_infer"
if os.path.exists(temp_frames_dir):
    for f in os.listdir(temp_frames_dir):
        os.remove(os.path.join(temp_frames_dir, f))
else:
    os.makedirs(temp_frames_dir)

cap = cv2.VideoCapture(test_video_path)
frame_idx = 0
while True:
    ret, frame = cap.read()
    if not ret:
        break
    save_path = os.path.join(temp_frames_dir, f"frame_{frame_idx:06d}.jpg")
    cv2.imwrite(save_path, frame)
    frame_idx += 1
cap.release()
print(f"Extracted {frame_idx} frames to '{temp_frames_dir}'")

# (C) TempFrameDataset 정의 및 DataLoader 구성
transform = get_ffpp_transforms(cfg["input_size"])
frame_paths = sorted([
    os.path.join(temp_frames_dir, fn)
    for fn in os.listdir(temp_frames_dir)
    if fn.endswith(".jpg") or fn.endswith(".png")
])

class TempFrameDataset(torch.utils.data.Dataset):
    def __init__(self, paths, transform):
        self.paths = paths
        self.transform = transform

    def __len__(self):
        return len(self.paths)

    def __getitem__(self, idx):
        img = Image.open(self.paths[idx]).convert("RGB")
        return self.transform(img), self.paths[idx]

temp_dataset = TempFrameDataset(frame_paths, transform)
temp_loader = DataLoader(
    temp_dataset,
    batch_size=cfg["batch_size"],
    shuffle=False,
    num_workers=cfg["num_workers"],
    pin_memory=True
)

# (D) 마지막 Fold 모델 로드
best_ckpt_path = os.path.join(cfg["output_dir"], f"fold_{cfg['training']['k_folds']}", "best.pth")
assert os.path.exists(best_ckpt_path), f"Checkpoint not found: {best_ckpt_path}"

model = DeepfakeAdapter(cfg).to(device)
ckpt = torch.load(best_ckpt_path, map_location=device)
model.load_state_dict(ckpt["model_state_dict"])
model.eval()

# (E) 프레임 단위 예측
results = {}  # { "frame_path": fake_probability }
with torch.no_grad():
    for imgs, paths in temp_loader:
        imgs = imgs.to(device)
        logits = model(imgs)
        probs = torch.softmax(logits, dim=1)[:, 1].cpu().numpy()  # fake 확률
        for p, prob in zip(paths, probs):
            results[p] = float(prob)

# (F) JSON으로 저장
video_name = os.path.splitext(os.path.basename(test_video_path))[0]
output_json_dir = "outputs/predictions"
os.makedirs(output_json_dir, exist_ok=True)
output_json_path = os.path.join(output_json_dir, f"{video_name}.json")

with open(output_json_path, "w") as f:
    json.dump(results, f, indent=2)

print(f"Saved prediction results to '{output_json_path}'")

AssertionError: Video not found: ../data/raw_videos/sample_video.mp4

## 결과 시각화 예시

- **추출된 프레임 중 상위 5개**를 선택하여,  
  프레임 원본 이미지 + 예측된 “Fake 확률”을 함께 시각화해 봅니다.
- JSON 파일에서 상위 5개 확률 프레임을 골라서 확인합니다.


In [ ]:
# 셀 6: 결과 시각화 예시

with open(output_json_path, "r") as f:
    results_json = json.load(f)

# 프레임별 확률 내림차순 정렬 → 상위 5개
sorted_items = sorted(results_json.items(), key=lambda x: x[1], reverse=True)
top5 = sorted_items[:5]

fig, axes = plt.subplots(1, 5, figsize=(20, 4))
for i, (frame_path, prob) in enumerate(top5):
    img = Image.open(frame_path).convert("RGB")
    axes[i].imshow(np.array(img))
    axes[i].set_title(f"{os.path.basename(frame_path)}\nFake: {prob:.3f}")
    axes[i].axis("off")
plt.tight_layout()
plt.show()


## 마무리 및 토의

1. **데이터 로드 및 전처리 확인**  
   - `FFPPFrameDataset`을 통해 **alltype** 폴더 구조에서 모든 프레임과 Real/Fake 레이블을 올바르게 로드했음을 확인했습니다.

2. **Train(70%) / Val(10%) / Test(20%) 분할**  
   - 전체 데이터를 80:20으로 분할한 뒤, Train(80%) 내부를 다시 7:1로 분할하여 **Train 70%, Val 10%, Test 20%** 비율을 만들었습니다.

3. **5-Fold Cross-Validation 학습 + 검증**  
   - `KFold(n_splits=5, shuffle=True, random_state=42)`로 Train(70%)을 5개 Fold로 나누어,  
   - 각 Fold마다 학습 및 검증을 수행하고 **Fold별 최적 모델(best.pth)** 을 저장했습니다.

4. **Test(20%) 최종 평가**  
   - 마지막(5번째) Fold의 최적 모델을 불러와 Test(20%) 데이터셋에 대해 **AUC/EER**을 계산했습니다.

5. **추론(Inference)**  
   - 새로운 비디오 파일을 “영상 → 프레임”으로 분할한 뒤,  
   - 마지막 Fold 모델로 프레임별 Deepfake 확률을 예측하여 JSON(`outputs/predictions/[video_name].json`)으로 저장했습니다.

6. **결과 시각화**  
   - 상위 5개 “Fake 확률이 높은 프레임”을 시각화하여,  
     모델이 어느 순간 가장 Fake이라고 판단했는지 직관적으로 확인했습니다.

---

### 향후 방향

- **하이퍼파라미터 튜닝**  
  - `training.k_folds`, `optimizer.lr`, `scheduler.T_max`, `model.adapter_dim` 등을 변경하며 성능 최적화  
  - 5-Fold 학습 결과를 앙상블(Ensembling)하여 Test 성능 향상 시도 가능

- **Transformer-XAI**  
  - 현재는 ViT + Dual-Level Adapter 기반 분류기까지만 구현되어 있습니다.  
  - 이후 “Attention Map”, “Token Attribution” 등 Transformer 설명 기법을 추가하여,  
    모델의 의사결정 근거를 시각화할 예정입니다.

- **배포 및 서비스**  
  학습된 모델(`fold_i/best.pth`)을 REST API 또는 Streamlit 앱으로 배포하여,  
  사용자가 업로드한 동영상에 대해 실시간으로 Deepfake 여부를 확인할 수 있도록 서비스화할 수 있습니다.

---

이 노트북이 FFPP 기반 DeepFake Detection 전체 파이프라인을 이해하고, 직접 실험해보는 데 도움이 되길 바랍니다. 감사합니다!
